
# Empirical Shift via \( \Delta^\*(B) \) from Monte Carlo \( H(B) \)

This notebook:
1. Samples \(n=120\) points \(B_i \in \mathbb R^{20}\) i.i.d. from \(\mathcal N(0.05\cdot \mathbf 1,\, 0.01^2 I)\) and sets \(P_0=P_n\) (the empirical law).
2. Defines
\[
L_t(x,z) = \exp\!\Big( (\sigma^{-1}(x-r\mathbf 1))^\top z - \tfrac12 \lVert \sigma^{-1}(x-r\mathbf 1)\rVert^2\, t \Big).
\]
3. Computes
\[
H(B) = \int_{\mathbb R^d} \nabla_b L_T(B,y)\; \Big(\mathbb E^{P_0}[L_T(B,y)]\Big)^{\frac{\alpha}{1-\alpha}} \varphi_T(y)\,dy
= \mathbb E_{Y\sim \mathcal N(0, TI)}\!\left[ \nabla_b L_T(B,Y)\,M(Y)\right]
\]
with Monte Carlo over \(Y\), where \(M(y)=\big(\mathbb E^{P_0}[L_T(B,y)]\big)^{\frac{\alpha}{1-\alpha}}\).
4. Forms
\[
\Delta^\*(B) = -\frac{\sqrt{\delta}\,H(B)}{\sqrt{\mathbb E^{P_0}\big[\lVert H(B)\rVert_2^2\big]}},
\qquad C_i = B_i + \Delta^\*(B_i),
\]
and returns the empirical law of \(\{C_i\}\).

All steps use vectorized NumPy for speed and numerical stability (log-mean-exp where helpful).


In [ ]:

# --- Parameters (edit as needed) ---
import numpy as np

rng = np.random.default_rng(12345)  # reproducible

# Dimensions / counts
d = 20           # dimension
n = 120          # number of samples for P0 = Pn
m = 200          # Monte Carlo draws for Y in the y-integral

# Model parameters
mu_B = 0.05      # each entry of mean vector for B
std_B = 0.01     # diag std dev for B
r = 0.03         # risk-free rate
T = 1.0          # time horizon
alpha = -1.0     # user's setting
delta = 0.006    # ambiguity radius used in Delta*

# Volatility / diffusion matrix sigma (invertible).
# Default: Identity. You can replace with any invertible matrix.
sigma_matrix = np.eye(d, dtype=np.float64)
sigma_inv = np.linalg.inv(sigma_matrix)

# Ones vector
ones = np.ones(d, dtype=np.float64)

# Exponent for M(y)
expo = alpha / (1.0 - alpha)  # for alpha = -1 this equals -1/2
expo


-0.5

In [2]:

# --- Sample B_1,...,B_n from N(0.05*1, 0.01^2 * I) ---
B = rng.normal(loc=mu_B, scale=std_B, size=(n, d)).astype(np.float64)

# --- Monte Carlo Y ~ N(0, T I) for the y-integral ---
Y = rng.normal(loc=0.0, scale=np.sqrt(T), size=(m, d)).astype(np.float64)

# Utility: stable log-sum-exp and log-mean-exp
def logsumexp(a, axis=None):
    a = np.asarray(a, dtype=np.float64)
    amax = np.max(a, axis=axis, keepdims=True)
    out = amax + np.log(np.sum(np.exp(a - amax), axis=axis, keepdims=True))
    if axis is None:
        return out.squeeze()
    return np.squeeze(out, axis=axis)

def logmeanexp(a, axis=None):
    a = np.asarray(a, dtype=np.float64)
    lse = logsumexp(a, axis=axis)
    if axis is None:
        N = a.size
    else:
        N = a.shape[axis]
    return lse - np.log(N)


In [5]:

# --- Precompute v_i = sigma^{-1} (B_i - r*1) for all i ---
# We store row-wise as v_i^T for convenient matrix multiplies.
V = (B - r*ones) @ sigma_inv.T                      # shape (n, d)
V_norm2 = np.sum(V*V, axis=1)                       # shape (n,)

# --- Compute log L_T(B_i, Y_j) = V_i dot Y_j - 0.5 * ||V_i||^2 * T ---
logL = V @ Y.T - 0.5 * (V_norm2[:, None]) * T       # shape (n, m)

# --- Compute M(Y_j) = ( E_{P0}[L_T(B, Y_j)] )^(alpha/(1-alpha)) ---
# Use log-mean-exp over i to avoid numerical issues.
log_mean_L_over_i = logmeanexp(logL, axis=0)        # shape (m,)
M = np.exp(expo * log_mean_L_over_i)                # shape (m,)
M.shape


(200,)

In [6]:

# Gradient: ∇_b L_T(B_i, Y_j) = L_T(B_i, Y_j) * (sigma^{-1})^T * (Y_j - T * v_i).
# Hence H(B_i) = E_Y[ ∇_b L_T(B_i, Y) * M(Y) ] = (sigma^{-1})^T * E_Y[ L * M * (Y - T v_i) ]
# We'll compute S_i := E_Y[ L * M * (Y - T v_i) ] row-wise, then H_i = S_i @ sigma_inv.

# Weights W_{i,j} = (1/m) * L_T(B_i, Y_j) * M(Y_j)
W = np.exp(logL) * (M[None, :]) / float(m)          # shape (n, m)

# First term: sum_j W_{i,j} * Y_j  -> (n, d)
WY = W @ Y                                          # (n, d)

# Row-sum s_i = sum_j W_{i,j}
s = np.sum(W, axis=1)                               # (n,)

# Second term: T * s_i * v_i  -> (n, d)   (note v_i stored as rows in V)
S = WY - (T * s)[:, None] * V                       # (n, d)

# H_i = S_i @ sigma_inv
H = S @ sigma_inv                                   # (n, d)

# Denominator: sqrt( E^{P0}[ ||H(B)||_2^2 ] ) = sqrt( (1/n) * sum_i ||H_i||^2 )
H_norm2 = np.sum(H*H, axis=1)                       # (n,)
denom = np.sqrt(np.mean(H_norm2))

denom


np.float64(0.27604443288885216)

In [ ]:

# --- Function: compute Delta^*(B) ---
import numpy as np

def compute_big_delta_star(B, r, T, alpha, delta, sigma_matrix, m=200, rng=None):
    """
    Compute the empirical-shift direction Delta^*(B) using Monte Carlo over Y ~ N(0, T I).

    Parameters
    ----------
    B : array-like, shape (n, d)
        Input samples representing the empirical law P0 = Pn.
    r : float
        Risk-free rate.
    T : float
        Time horizon.
    alpha : float
        Robustness parameter.
    delta : float
        Ambiguity radius used in Delta*.
    sigma_matrix : array-like, shape (d, d)
        Invertible diffusion matrix.
    m : int, optional (default=200)
        Number of Monte Carlo draws for Y.
    rng : numpy.random.Generator, optional
        Random number generator for reproducibility. If None, a new Generator() is used.

    Returns
    -------
    Delta : ndarray, shape (n, d)
        The Delta^*(B) shifts for each row of B.
    """
    B = np.asarray(B, dtype=np.float64)
    if B.ndim != 2:
        raise ValueError("B must be a 2D array of shape (n, d)")
    n, d = B.shape

    if rng is None:
        rng = np.random.default_rng()

    sigma_matrix = np.asarray(sigma_matrix, dtype=np.float64)
    sigma_inv = np.linalg.inv(sigma_matrix)

    ones = np.ones(d, dtype=np.float64)
    expo = alpha / (1.0 - alpha)

    # Local stable log-sum-exp utilities
    def _logsumexp(a, axis=None):
        a = np.asarray(a, dtype=np.float64)
        amax = np.max(a, axis=axis, keepdims=True)
        out = amax + np.log(np.sum(np.exp(a - amax), axis=axis, keepdims=True))
        if axis is None:
            return out.squeeze()
        return np.squeeze(out, axis=axis)

    def _logmeanexp(a, axis=None):
        a = np.asarray(a, dtype=np.float64)
        lse = _logsumexp(a, axis=axis)
        if axis is None:
            N = a.size
        else:
            N = a.shape[axis]
        return lse - np.log(N)

    # Monte Carlo draws Y ~ N(0, T I)
    Y = rng.normal(loc=0.0, scale=np.sqrt(T), size=(m, d)).astype(np.float64)

    # v_i = sigma^{-1} (B_i - r*1)
    V = (B - r * ones) @ sigma_inv.T                  # (n, d)
    V_norm2 = np.sum(V * V, axis=1)                   # (n,)

    # log L_T(B_i, Y_j)
    logL = V @ Y.T - 0.5 * (V_norm2[:, None]) * T     # (n, m)

    # M(Y_j) = ( E_{P0}[L_T(B, Y_j)] )^(alpha/(1-alpha))
    log_mean_L_over_i = _logmeanexp(logL, axis=0)     # (m,)
    M = np.exp(expo * log_mean_L_over_i)              # (m,)

    # H(B_i) computation via weights
    W = np.exp(logL) * (M[None, :]) / float(m)        # (n, m)
    WY = W @ Y                                        # (n, d)
    s = np.sum(W, axis=1)                             # (n,)
    S = WY - (T * s)[:, None] * V                     # (n, d)
    H = S @ sigma_inv                                 # (n, d)

    # Denominator: sqrt( E^{P0}[ ||H(B)||_2^2 ] )
    H_norm2 = np.sum(H * H, axis=1)
    denom = np.sqrt(np.mean(H_norm2))

    if denom == 0.0 or not np.isfinite(denom):
        Delta = np.zeros_like(B)
    else:
        Delta = np.sqrt(delta) * H / denom

    return Delta



In [ ]:

# --- Delta^*(B_i) via function and shifted samples C_i ---
Delta = compute_delta_star(
    B=B,
    r=r,
    T=T,
    alpha=alpha,
    delta=delta,
    sigma_matrix=sigma_matrix,
    m=m,
    rng=rng,
)

C = B + Delta

# Basic sanity checks
B_mean = B.mean(axis=0)
C_mean = C.mean(axis=0)
B_std = B.std(axis=0, ddof=1)
C_std = C.std(axis=0, ddof=1)

print("Shapes: B", B.shape, "Delta", Delta.shape, "C", C.shape)
print("Mean(B)[:5]:", B_mean[:5])
print("Mean(C)[:5]:", C_mean[:5])
print("Std(B) (per-dim) first 5:", B_std[:5])
print("Std(C) (per-dim) first 5:", C_std[:5])


Shapes: B (120, 20) H (120, 20) Delta (120, 20) C (120, 20)
||H||_2 (avg over i): 0.27604443288885216
Mean(B)[:5]: [0.04814551 0.04940982 0.05114938 0.05112614 0.04962243]
Mean(C)[:5]: [0.07636861 0.03164736 0.0709019  0.04112399 0.03077643]
Std(B) (per-dim) first 5: [0.00955588 0.0104546  0.0104167  0.01183611 0.00879081]
Std(C) (per-dim) first 5: [0.00938612 0.0105131  0.01088511 0.01186915 0.00900725]


In [10]:
Delta

array([[ 0.02895747, -0.01770298,  0.01894733, ...,  0.01114293,
         0.02512816,  0.0165398 ],
       [ 0.02746001, -0.01797975,  0.02026449, ...,  0.00929966,
         0.02394371,  0.01600569],
       [ 0.02928585, -0.01836085,  0.02085612, ...,  0.01026428,
         0.0244544 ,  0.0152185 ],
       ...,
       [ 0.02698868, -0.01771915,  0.02204272, ...,  0.00975391,
         0.02423734,  0.01311033],
       [ 0.02776239, -0.01898411,  0.02003331, ...,  0.01034894,
         0.02515572,  0.01546593],
       [ 0.02905986, -0.01818727,  0.02000468, ...,  0.00927049,
         0.02372911,  0.01710059]])


### Notes on \(\sigma\)

- This notebook defaults to \(\sigma=I_d\). To use a general invertible \(\sigma\), set:
  ```python
  sigma_matrix = <your dxd invertible matrix>
  sigma_inv = np.linalg.inv(sigma_matrix)
  ```
- All formulas adapt automatically; we only ever use \(\sigma^{-1}\) and \((\sigma^{-1})^\top\).
